# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Wednesday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**


# Question 1 (40 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]

(3) Collect all the reviews of the top 1000 most popular software from G2 or Capterra.

(4) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(5) Collect all the information of the 904 narrators in the Densho Digital Repository.


In [7]:
import requests
from bs4 import BeautifulSoup
import csv


movie_id = "11274536"

reviews = []
count = 1
reviews_gained = 0

while reviews_gained < 1000:
    base_url = f"https://www.imdb.com/title/{movie_id}/reviews"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

    url = f"{base_url}?sort=submissionDate&dir=desc&ratingFilter=0&page={count}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    review_containers = soup.find_all('div', class_='review-container')

    if not review_containers:
        break

    for container in review_containers:
        review_containing_text = container.find('div', class_='text').text.strip()
        reviews.append(review_containing_text)
        reviews_gained += 1
        if reviews_gained == 1000:
            break

    count += 1

with open('oppenheimer_reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Review'])
    for review in reviews:
        writer.writerow([review])

print("Data Saved to csv file")


Data Saved to csv file


# Question 2 (30 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [23]:
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# IMDB movie ID for the movie "Oppenheimer"
movie_id = "783465u85t5"

# Load NLTK components
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to clean text
def clean_text(text):
    # Remove noise: special characters and punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Lowercase all texts
    text = text.lower()
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    # Stemming
    text = ' '.join(stemmer.stem(word) for word in text.split())
    # Lemmatization
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return text

# Read the original reviews from the CSV file
original_reviews = []
with open('oppenheimer_reviews.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        original_reviews.append(row[0])

# Clean the reviews
cleaned_reviews = [clean_text(review) for review in original_reviews]

# Write the cleaned reviews to a new CSV file with a new column for clean text
with open('clean_oppenheimer_reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Original Review', 'Cleaned Review'])
    for original_review, cleaned_review in zip(original_reviews, cleaned_reviews):
        writer.writerow([original_review, cleaned_review])

print("Data Saved to the csv")


Data Saved to the csv


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Question 3 (30 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [24]:
import csv
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.parse import CoreNLPParser, CoreNLPDependencyParser
from collections import Counter

# Load the clean text from the CSV file
clean_text = []
with open('clean_oppenheimer_reviews.csv', 'r', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        clean_text.append(row[1])  # Assuming clean text is in the second column

# Combine all the reviews into a single string
clean_text_combined = ' '.join(clean_text)

# (1) Parts of Speech (POS) Tagging
tokens = word_tokenize(clean_text_combined)
pos_tags = pos_tag(tokens)

noun_count = len([word for word, pos in pos_tags if pos.startswith('NN')])
verb_count = len([word for word, pos in pos_tags if pos.startswith('VB')])
adj_count = len([word for word, pos in pos_tags if pos.startswith('JJ')])
adv_count = len([word for word, pos in pos_tags if pos.startswith('RB')])

print("Parts of Speech (POS) Tagging:")
print("Noun Count:", noun_count)
print("Verb Count:", verb_count)
print("Adjective Count:", adj_count)
print("Adverb Count:", adv_count)

# (2) Constituency Parsing and Dependency Parsing
parser = CoreNLPParser(url='http://localhost:9000')
dep_parser = CoreNLPDependencyParser(url='http://localhost:9000')

print("\nConstituency Parsing Trees:")
for sentence in sent_tokenize(clean_text_combined):
    tree = next(parser.raw_parse(sentence))
    print(tree)

print("\nDependency Parsing Trees:")
for sentence in sent_tokenize(clean_text_combined):
    tree = next(dep_parser.raw_parse(sentence))
    print(tree.tree())

# (3) Named Entity Recognition
named_entities = []

for sentence in sent_tokenize(clean_text_combined):
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)
    chunked_words = ne_chunk(tagged_words)
    named_entities.extend(chunked_words)

# Count entities
entity_counter = Counter()
for entity in named_entities:
    if isinstance(entity, tuple):
        entity_counter[entity[1]] += 1

print("\nNamed Entity Recognition:")
print(entity_counter)





LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

In [ ]:
The assignment was very tough and finding the code for this is really challenging. I  have learnt new things in Python Programming.